In [115]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [167]:
import numpy as np
import pandas as pd
import datetime as dt
import statistics
import collections

# Reflect Tables into SQLAlchemy ORM

In [168]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, Column, Integer, Date, VARCHAR, Text
from sqlalchemy import *
Base = automap_base()


In [169]:
class measurement(Base):
    __tablename__ = 'measurement'
    id = Column(VARCHAR, primary_key=True)
    station = Column(VARCHAR)
    date = Column(Date)
    prcp = Column(Integer)
    tobs = Column(Integer)

In [170]:
class station(Base):
    __tablename__ = 'station'
    id = Column(VARCHAR, primary_key=True)
    station = Column(VARCHAR)
    name = Column(VARCHAR)
    latitude = Column(Integer)
    longitude = Column(Integer)
    elevation = Column(Integer)

In [171]:
Base.metadata.tables

immutabledict({'measurement': Table('measurement', MetaData(bind=None), Column('id', VARCHAR(), table=<measurement>, primary_key=True, nullable=False), Column('station', VARCHAR(), table=<measurement>), Column('date', Date(), table=<measurement>), Column('prcp', Integer(), table=<measurement>), Column('tobs', Integer(), table=<measurement>), schema=None), 'station': Table('station', MetaData(bind=None), Column('id', VARCHAR(), table=<station>, primary_key=True, nullable=False), Column('station', VARCHAR(), table=<station>), Column('name', VARCHAR(), table=<station>), Column('latitude', Integer(), table=<station>), Column('longitude', Integer(), table=<station>), Column('elevation', Integer(), table=<station>), schema=None)})

In [172]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///../sqlalchemy-challenge/resources/hawaii.sqlite", echo=False)
conn = engine.connect

In [173]:
Base.metadata.create_all(engine)

In [174]:
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [175]:
inspector = inspect(engine)
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c['type'])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [176]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [177]:
# View all of the classes that automap found
Base.classes.keys()

['Measurements', 'Stations', 'measurement', 'station']

In [178]:
# Save references to each table
station = Base.classes.station
measurement = Base.classes.measurement

In [179]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [180]:
# Find the most recent date in the data set.
recent = session.query(measurement.date).order_by(measurement.date.desc()).first()
print(recent)

('2017-08-23',)


In [181]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database
    
result = session.query(measurement.date, func.sum(measurement.prcp)).\
    filter(measurement.date > '2016-08-23').\
    group_by(measurement.date).\
    order_by(func.sum(measurement.date).desc()).all()
result

# Use Pandas Plotting with Matplotlib to plot the data




[('2017-07-31', 0.0),
 ('2017-07-28', 0.77),
 ('2017-07-27', 0.01),
 ('2017-07-26', 0.5),
 ('2017-07-25', 0.61),
 ('2017-07-21', 0.13),
 ('2017-07-20', 1.24),
 ('2017-07-18', 2.53),
 ('2017-07-13', 1.81),
 ('2017-07-12', 0.36000000000000004),
 ('2017-07-11', 0.03),
 ('2017-07-10', 0.04),
 ('2017-07-07', 0.5),
 ('2017-07-05', 0.0),
 ('2017-07-03', 0.74),
 ('2017-06-30', 0.52),
 ('2017-06-29', 0.07),
 ('2017-06-26', 0.1),
 ('2017-06-23', 0.6699999999999999),
 ('2017-06-19', 0.73),
 ('2017-06-16', 0.2),
 ('2017-06-15', 2.71),
 ('2017-06-14', 1.57),
 ('2017-06-13', 1.38),
 ('2017-06-08', 0.03),
 ('2017-06-07', 0.01),
 ('2017-06-06', 0.0),
 ('2017-06-05', 0.08),
 ('2017-04-28', 4.24),
 ('2017-04-27', 0.42),
 ('2017-04-25', 0.0),
 ('2017-04-24', 0.060000000000000005),
 ('2017-04-21', 8.38),
 ('2017-04-20', 0.78),
 ('2017-04-19', 0.2),
 ('2017-04-18', 3.36),
 ('2017-04-17', 3.0700000000000003),
 ('2017-04-13', 1.09),
 ('2017-04-12', 1.08),
 ('2017-04-11', 0.47),
 ('2017-04-10', 0.02),
 ('2017

In [182]:
# Calculate the date one year from the last date in data set.
query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print("Query Date: ", query_date)

Query Date:  2016-08-23


In [183]:
# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(result, columns = ['date', 'prcp'])

print(df)


           date  prcp
0    2017-07-31  0.00
1    2017-07-28  0.77
2    2017-07-27  0.01
3    2017-07-26  0.50
4    2017-07-25  0.61
..          ...   ...
360  2017-08-22  0.50
361  2017-08-20  0.01
362  2017-08-12  0.14
363  2017-08-09  0.15
364  2017-08-05  0.06

[365 rows x 2 columns]


In [184]:
# Sort the dataframe by date
df = df.sort_values(by="date")
df

,date,prcp
146,2016-08-24,9.33
145,2016-08-25,0.54
283,2016-08-26,0.10
282,2016-08-27,0.32
281,2016-08-28,3.10
...,...,...
339,2017-08-19,0.09
361,2017-08-20,0.01
338,2017-08-21,0.58
360,2017-08-22,0.50


In [185]:
print(statistics.pvariance(df["prcp"]))
print(statistics.variance(df["prcp"]))
print(statistics.pstdev(df['prcp']))
print(statistics.stdev(df["prcp"]))

3.147184849690374
3.155830961914798
1.7740306789033762
1.776465862862216


In [186]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df[["prcp"]].describe()

,prcp
count,365.000000
mean,0.974164
std,1.776466
min,0.000000
25%,0.050000
50%,0.400000
75%,1.080000
max,14.280000


# Exploratory Station Analysis

In [187]:
# Design a query to calculate the total number stations in the dataset
total_stations = session.query(station).count()
total_stations



9

In [217]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.

active = session.query(measurement).value_counts()



AttributeError: 'Query' object has no attribute 'value_counts'

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()